#### Monte Carlo Simulation

In [94]:
import numpy as np   
import pandas as pd    
import random  as rd
from itertools import combinations , product

In [112]:
 class Die:
    '''
    PURPOSE:
    A class accepts variety of random variables associated with stochastic
    processes and rolls/runs and returns a list of outcomes
    
    ATTRIBUTES:
    Takes an array of faces 
    
    METHODS:
    __init__::Return a dataframe with faces and weights column
    change_weight:: Changes the weight of a single side/face.
    roll_die:: Roll/Run the die one or more times
    show_state:: Display the current set of the faces and weights
    -------------------------------------------------------------------------
    '''
    def __init__(self , faces):
        '''
        PURPOSE:
        Initializes private dataframe containing faces and weights respectively
    
        INPUTS:
        Takes an array of faces 
    
        OUTPUTS:
        Return a dataframe with faces and weights column
        '''
        self.faces = list(set(faces)) # The faces must be unique
        self.weights= np.ones(len(faces))  # Initialize the weight to 1.0
        self.faces_weights_df= pd.DataFrame(self.faces , columns=['faces'])
        self.faces_weights_df = self.faces_weights_df.assign(weights=self.weights)
    def change_weight(self , face_value , new_weight):
        # face passed must be valid
        is_face_valid = (lambda face_value: face_value in self.faces) 
        if(not is_face_valid(face_value)): return "Error:The face passed is invalid."
        # Weight passed must be valid
        is_weight_valid = isinstance(new_weight, float) | isinstance(new_weight, int) | isinstance(new_weight, bool)
        if(not is_weight_valid): return "Error:The Weight passed is invalid."
        self.faces_weights_df.iloc[self.faces.index(face_value) , [1]] = new_weight
    def roll_die(self , number_of_rolls = 1):
         return [rd.choice(self.faces) for roll in range(number_of_rolls)]
    def show_state(self):
        return self.faces_weights_df

In [150]:
number_die_array = [1, 2 ,3 ,4 ,5 ,6]
string_coin_array = ['Heads', 'Tails']
float_die_array = [1.0 , 2.0 , 3.0 , 4.0 , 5.0 , 6.0]
d = s = v = Die(float_die_array)
d.show_state()
d.change_weight(2, 2.0)
d.change_weight(3, 3)
d.change_weight(4, 4)
d.change_weight(5, 5)
d.change_weight(6, 6)
d.roll_die(10)

[1.0, 4.0, 5.0, 4.0, 4.0, 1.0, 5.0, 4.0, 4.0, 4.0]

In [20]:
Die.__doc__

'\n   PURPOSE:\n   A class accepts variety of random variables associated with stochastic\n   processes and rolls/runs and returns a list of outcomes\n   \n   ATTRIBUTES:\n   Takes an array of faces \n   \n   METHODS:\n   __init__::Return a dataframe with faces and weights column\n   change_weight:: Changes the weight of a single side/face.\n   roll_die:: Roll/Run the die one or more times\n   show_state:: Display the current set of the faces and weights\n   -------------------------------------------------------------------------\n   '

In [152]:
class Game: # Or access using Game(Die)
    def __init__(self , dice):
        self.dice = dice  # Taking the list of dice
    def play(self, number_of_rolls):
        self.number_of_rolls = number_of_rolls
        self.play_result_df_list = []
        # Iterate through the list of dice and perform the same operations 
        for roll in self.dice:
            self.cols = [str(die) for die in roll.faces]
            roll_results = [roll for roll in roll.roll_die(self.number_of_rolls)] # Roll the dice X number of times extract the results  
            # Create a single dim array with similar results populate across the array
            roll_data = np.array([np.full((1 ,len(roll.faces)), roll_result)[0] for roll_result in roll_results]) 
            index = [i for i in range(1, len(roll_data) + 1)]
            self.play_df = pd.DataFrame(roll_data ,
                                   index = index,
                                   columns = self.cols)  
            self.play_df.index.name = 'roll number' 
            self.show(self.play_df , 1)
        
    
    def show(self , play_result_df , df_form = 1): # default df return form is Wide = 1 and Narrow option is 2
        if (df_form != 1 and df_form != 2):
            return print("Error:The dataframe display format option can only be Wide(value 1) or Narrow(value 2).")
        
        self.play_result_df_list.append(play_result_df)
        if df_form == 1:
            return play_result_df
        else:
            return pd.melt(panda_df, value_vars=self.cols, var_name='VARS', value_name='VALUES', ignore_index=False)
        
dice = []
dice.extend([s , d, v]) # Playing a Two dice 
g = Game(dice)
g.play(10)

In [161]:
# An analyzer takes the results of a single game and computes statistical properties about it.
class Analyzer:
     def __init__(self , game):
            self.game = game
            self.df_data_type = type(game.dice[0].faces[1]) # infers the data type of the die faces  
            self.game_result = game.play_result_df_list
            
     # method to compute how many times a given face is rolled in each event       
     def face_count(self):
        index = [i for i in range(1, self.game.number_of_rolls + 1)] 
        self.face_count_df = pd.DataFrame(index=index , columns = self.game.cols) 
        self.face_count_df.index.name = 'roll number'
        # Iterate through the resulting list array and extract each parallel value to match and increment properly
        for parallel_index in range(self.game.number_of_rolls):
            start = 0
            self.face_array = np.zeros(len(self.game.cols), dtype = int) # Hold the face value and increment when new similar value is found uses Game Cols Structure Copy
            while start < len(self.game_result): 
                self.face_array[self.game.cols.index(str(self.game_result[start].iloc[parallel_index][self.game.cols[0]]))] = self.face_array[self.game.cols.index(str(self.game_result[start].iloc[parallel_index][self.game.cols[0]]))] + 1
                start += 1
            # Append the new face counts to the dataframe 
            self.face_count_df.loc[parallel_index + 1] = self.face_array
        
        return self.face_count_df
    
     # method to compute how many times the game resulted in all faces being identical 
     def jackpot(self):
        self.jackpot_results_df = pd.DataFrame(columns = self.game.cols) 
        self.jackpot_results_df.index.name = 'roll number'
        self.jack_pot_indices = []
        # Iterate through the resulting list array and extract each parallel value to match and increment properly
        for parallel_index in range(self.game.number_of_rolls):
            start = 0
            self.jackpot_array = np.zeros(len(self.game.cols), dtype = int) # Hold the face value and increment when new similar value is found uses Game Cols Structure Copy
            while start < len(self.game_result): 
                self.jackpot_array[self.game.cols.index(str(self.game_result[start].iloc[parallel_index][self.game.cols[0]]))] = self.jackpot_array[self.game.cols.index(str(self.game_result[start].iloc[parallel_index][self.game.cols[0]]))] + 1
                start += 1
                # Save the index only if all values result in similar faces 
                if len(self.game.dice) in self.jackpot_array: 
                    # Append the Index for the dataframe
                    self.jack_pot_indices.append(parallel_index + 1)
                    # Append the new face counts to the jackpot_results dataframe
                    self.jackpot_results_df.loc[parallel_index + 1] = self.jackpot_array
        
        return len(self.jack_pot_indices)
       
     # Combo method to compute the distinct/unique combinations of faces rolled, along with their counts
    # Count is always 1 because the specific combination only occurs Once
     def combo(self): 
        # Loop through the results array and add to the rolled_face_list if not present
#         print(f"Result-> {self.game_result}") 
        face_combination_list = [] 
        # Check if the face has been rolled during the Game , checking presence
        for roll in self.game_result:
            for i in self.game.cols: 
                # Check the type of Dataframe type to conditionally pass casting
                if self.df_data_type == int:
                    check_face_presence = (roll.loc[roll[self.game.cols[0]]==int(i)]) 
                elif self.df_data_type == str:
                    check_face_presence = (roll.loc[roll[self.game.cols[0]]==i])
                else:
                    check_face_presence = (roll.loc[roll[self.game.cols[0]]==float(i)])
                # Append to the present list if this face has been rolled during the Game
                if (not check_face_presence.empty): face_combination_list.append(i)
#                 print(f"found-> {check_face_presence.empty }")
        # Extract only the Unique values 
        face_combination_list = list(set(face_combination_list)) 
        
        # Sort the values
        face_combination_list.sort(key = self.df_data_type)
        
#         print(f"Final->{type(face_combination_list)}")
        # Perform the Combination of unique values and store it in combination_df for public access
        # initializing the range with the Number of Dice rolled 
        # using list comprehension to formulate all distinct combinations of the elements
#         combo_list = [list(face_combination_list) for _ in range(len(self.game.dice))]

        # using product() to get Combinations
#         combination_list = list(product(*combo_list))
        print(f"face_combination_list->{face_combination_list}")
        print(f"COs->{len(self.game.dice)}")
        # Initialize the com
        combination_list = []
        # If the List type of str the different combinations reseambles the permutation
        if self.df_data_type == str:
            # using list comprehension to formulate all distinct combinations of the elements
            combo_list = [list(face_combination_list) for _ in range(len(self.game.dice))] 
            # using product() to get Combinations
            combination_list = list(product(*combo_list))
        else: 
            combination_list = list(combinations(face_combination_list, len(self.game.dice)))
       
        print(f"COMV->{combination_list}")
#         print("The constructed dice Combinations : " + str(combination_list))
        
        # Construct the combination tuples indices 
        combination_indice_list = [tuple([i] * len(combination_list)) for i in range(len(combination_list))]
        print(len(combination_indice_list))
        
        # Construct the Index name 
        combination_indice_name_list = ["roll_number "+str(index + 1) for index in range(len(combination_list))]
        print(len(combination_indice_name_list)) 
        
        # Create Row Level MultiIndex 
        combination_index = pd.MultiIndex.from_tuples(combination_indice_list, names=combination_indice_name_list)
        print(len(combination_index)) 
        
        # Create Column Level MultiIndex 
        cols = pd.MultiIndex.from_tuples(combination_list)
        print(f"COls => {len(cols)} , {cols}")
            
        # Create Data for the MultiIndex table 
        data = [[1 for i in range(len(combination_list))] for i in range(len(combination_list))]
        print(len(data))
        # Construct the Combination Dataframe 
        combination_df = pd.DataFrame(data, columns=cols,index=combination_index)
        return combination_df
    
     def permutation(self): 
        # Loop through the results array and add to the rolled_face_list if not present
#         print(f"Result-> {self.game_result}") 
        face_permutation_list = [] 
        # Check if the face has been rolled during the Game , checking presence
        for roll in self.game_result:
            for i in self.game.cols: 
                # Check the type of Dataframe type to conditionally pass casting
                if self.df_data_type == int:
                    check_face_presence = (roll.loc[roll[self.game.cols[0]]==int(i)]) 
                elif self.df_data_type == str:
                    check_face_presence = (roll.loc[roll[self.game.cols[0]]==i])
                else:
                    check_face_presence = (roll.loc[roll[self.game.cols[0]]==float(i)])
                # Append to the present list if this face has been rolled during the Game
                if (not check_face_presence.empty): face_permutation_list.append(i)
#                 print(f"found-> {check_face_presence.empty }")
        # Extract only the Unique values 
        face_permutation_list = list(set(face_permutation_list)) 
        
        # Sort the values
        face_permutation_list.sort(key = self.df_data_type)
        
#         print(f"Final->{type(face_permutation_list)}")
        # Perform the permutation  of unique values and store it in permutation_df for public access
        # initializing the range with the Number of Dice rolled 
        # using list comprehension to formulate all distinct permutation s of the elements
        perm_list = [list(face_permutation_list) for _ in range(len(self.game.dice))]

        # using product() to get permutation s
        permutation_list = list(product(*perm_list))
#         print("The constructed dice permutation s : " + str(permutation_list))
        
        # Construct the permutation  tuples indices 
        permutation_indice_list = [tuple([i] * len(permutation_list)) for i in range(len(permutation_list))]
        print(len(permutation_indice_list))
        
        # Construct the Index name 
        permutation_indice_name_list = ["roll_number "+str(index + 1) for index in range(len(permutation_list))]
        print(len(permutation_indice_name_list)) 
        
        # Create Row Level MultiIndex 
        permutation_index = pd.MultiIndex.from_tuples(permutation_indice_list, names=permutation_indice_name_list)
        print(len(permutation_index)) 
        
        # Create Column Level MultiIndex 
        cols = pd.MultiIndex.from_tuples(permutation_list)
        print(f"COls => {len(cols)} , {cols}")
            
        # Create Data for the MultiIndex table 
        data = [[1 for i in range(len(permutation_list))] for i in range(len(permutation_list))]
        print(len(data))
        # Construct the permutation  Dataframe 
        permutation_df = pd.DataFrame(data, columns=cols,index=permutation_index)
        return permutation_df


a = Analyzer(g)
#a.face_count()
#a.jackpot()
#a.combo()
#a.permutation()

In [23]:
# ["foo", "bar", "baz"].index("bar")
"bar" in ["foo", "bar", "baz"] 

True

In [24]:
# ['1','2','3','4','5','6'].index('3') 

In [25]:
for i in  range(5):
    print(i)

0
1
2
3
4


In [26]:
# index = [i for i in range(1, 1 + 1)]
index = [4 ,5]
df = pd.DataFrame(index=index , columns = ['1', '2', '3', '4', '5', '6'])
df
row = [1,0,0 ,1 ,0 ,1]
# index = [i for i in range(1, len(row) + 1)]
# cols = [1,2,3,4,5,6]
 
df.loc[4] = [20, 7, 5 , 3, 4, 6]
df.loc[5] = [20, 7, 15 ,233, 4, 6]
# df = df.dropna(axis=1, how='all')
df

,1,2,3,4,5,6
4,20,7,5,3,4,6
5,20,7,15,233,4,6


In [27]:
# Create Row Level MultiIndex 
new_index = pd.MultiIndex.from_tuples([("r0", "rA"),
                                       ("r1", "rB")],
                                       names=['indx1','indx2'])

# Create Column Level MultiIndex 
cols = pd.MultiIndex.from_tuples([("Gasoline", "Toyoto"), 
                                  ("Gasoline", "Ford"), 
                                  ("Electric", "Tesla"),
                                  ("Electric", "Nio")])

# Create MultiIndex DataFrame
data=[[100,300, 900,400 ], [200,500, 300,600]]
df = pd.DataFrame(data, columns=cols,index=new_index)
print(df)

            Gasoline      Electric     
              Toyoto Ford    Tesla  Nio
indx1 indx2                            
r0    rA         100  300      900  400
r1    rB         200  500      300  600


In [28]:
 from itertools import product
  
# initializing K Number of Dice rolled
K = 3
  
# using list comprehension to formulate elements
# temp = [list(range(1, 7)) for _ in range(K)]
temp = [list([1,3,5]) for _ in range(K)]
  
# using product() to get Combinations
res = list(product(*temp))
  
# printing result
print("The constructed dice Combinations : " + str(res))

The constructed dice Combinations : [(1, 1, 1), (1, 1, 3), (1, 1, 5), (1, 3, 1), (1, 3, 3), (1, 3, 5), (1, 5, 1), (1, 5, 3), (1, 5, 5), (3, 1, 1), (3, 1, 3), (3, 1, 5), (3, 3, 1), (3, 3, 3), (3, 3, 5), (3, 5, 1), (3, 5, 3), (3, 5, 5), (5, 1, 1), (5, 1, 3), (5, 1, 5), (5, 3, 1), (5, 3, 3), (5, 3, 5), (5, 5, 1), (5, 5, 3), (5, 5, 5)]


In [78]:
# Create Row Level MultiIndex 
new_index = pd.MultiIndex.from_tuples([("1", "1", "1"),
                                       ("2", "2", "2"),
                                       ("3", "3" , "3")],
                                       names=['indx1','indx2' , "indx3"])

# Create Column Level MultiIndex 
cols = pd.MultiIndex.from_tuples( [(1, 1, 1), (1, 1, 3), (1, 1, 5), (1, 3, 1), (1, 3, 3), (1, 3, 5), (1, 5, 1), (1, 5, 3), (1, 5, 5), (3, 1, 1), (3, 1, 3), (3, 1, 5), (3, 3, 1), (3, 3, 3), (3, 3, 5), (3, 5, 1), (3, 5, 3), (3, 5, 5), (5, 1, 1), (5, 1, 3), (5, 1, 5), (5, 3, 1), (5, 3, 3), (5, 3, 5), (5, 5, 1), (5, 5, 3), (5, 5, 5)])

print(new_index)
# Create MultiIndex DataFrame
data=[[1,1, 1, 1, 1, 1, 1,1 , 1,1, 1, 1, 1, 1, 1,1, 1,1, 1, 1, 1, 1, 1,1 ,1,1,1],[1,1, 1, 1, 1, 1, 1,1 , 1,1, 1, 1, 1, 1, 1,1, 1,1, 1, 1, 1, 1, 1,1 ,1,1,1], [1,1, 1, 1, 1, 1, 1,1 , 1,1, 1, 1, 1, 1, 1,1, 1,1, 1, 1, 1, 1, 1,1 ,1,1,1]]
# print(len(cols))
print(len(data))
df = pd.DataFrame(data, columns=cols,index=new_index)
print(df)

MultiIndex([('1', '1', '1'),
            ('2', '2', '2'),
            ('3', '3', '3')],
           names=['indx1', 'indx2', 'indx3'])
3
                   1                          3  ...     5                    \
                   1        3        5        1  ...  5  1        3        5   
                   1  3  5  1  3  5  1  3  5  1  ...  5  1  3  5  1  3  5  1   
indx1 indx2 indx3                                ...                           
1     1     1      1  1  1  1  1  1  1  1  1  1  ...  1  1  1  1  1  1  1  1   
2     2     2      1  1  1  1  1  1  1  1  1  1  ...  1  1  1  1  1  1  1  1   
3     3     3      1  1  1  1  1  1  1  1  1  1  ...  1  1  1  1  1  1  1  1   

                         
                         
                   3  5  
indx1 indx2 indx3        
1     1     1      1  1  
2     2     2      1  1  
3     3     3      1  1  

[3 rows x 27 columns]


In [30]:
res = tuple([2] * 6) # def_val the number to duplicate and N size of the data arrays 
res

(2, 2, 2, 2, 2, 2)

In [31]:
# from tuples cols 
combo_result = [(1, 1, 1), (1, 1, 3), (1, 1, 5), (1, 3, 1), (1, 3, 3), (1, 3, 5), (1, 5, 1), (1, 5, 3), (1, 5, 5), (3, 1, 1), (3, 1, 3), (3, 1, 5), (3, 3, 1), (3, 3, 3), (3, 3, 5), (3, 5, 1), (3, 5, 3), (3, 5, 5), (5, 1, 1), (5, 1, 3), (5, 1, 5), (5, 3, 1), (5, 3, 3), (5, 3, 5), (5, 5, 1), (5, 5, 3), (5, 5, 5)]

In [32]:
# from tuples indices 
res = [tuple([i] * len(combo_result)) for i in range(1 , len(combo_result))]
res

[(1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1),
 (2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2),
 (3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3),
 (4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4),
 (5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5),
 (6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6),
 (7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7),
 (8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8

In [33]:
# Index name
len(combo_result)
["roll_number "+str(index + 1) for index in range(len(combo_result))]
    


['roll_number 1',
 'roll_number 2',
 'roll_number 3',
 'roll_number 4',
 'roll_number 5',
 'roll_number 6',
 'roll_number 7',
 'roll_number 8',
 'roll_number 9',
 'roll_number 10',
 'roll_number 11',
 'roll_number 12',
 'roll_number 13',
 'roll_number 14',
 'roll_number 15',
 'roll_number 16',
 'roll_number 17',
 'roll_number 18',
 'roll_number 19',
 'roll_number 20',
 'roll_number 21',
 'roll_number 22',
 'roll_number 23',
 'roll_number 24',
 'roll_number 25',
 'roll_number 26',
 'roll_number 27']

In [46]:
#  DATA
print([ [1 for i in range(216)] for i in range(216)]) 

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [135]:
# diff = list(combinations([1,2,5,6], 3))
diff = list(combinations(['Heads', 'Tails'] , 3))
print(len(diff))
print(diff)
#[(1, 2, 3, 4), (1, 2, 3, 5), (1, 2, 4, 5), (1, 3, 4, 5), (2, 3, 4, 5)]

0
[]
